In [1]:
import sys
from Adafruit_IO import MQTTClient
import random
import time
import serial.tools.list_ports

In [2]:
AIO_FEED_ID_1 = "bb-humid"
AIO_FEED_ERROR_1 = "error-control1"
AIO_USERNAME = "bbace"
AIO_KEY = "aio_syWW38AYsrDDPbBMI4iLR05YPl8L"

In [3]:
def connected( client ):
    print ("Ket noi thanh cong ...")
    client.subscribe ( AIO_FEED_ID_1 )
    client.subscribe ( AIO_FEED_ERROR_1 )
def subscribe( client , userdata , mid , granted_qos ):
    print(" Subcribe thanh cong ...")

def disconnected( client ):
    print(" Ngat ket noi ...")
    sys.exit (1)

def message(client , feed_id , payload ):
    global sending_mess_again
    global failure
    global published
    global timer
    print(" Nhan du lieu : " +feed_id+" "+payload )
    if(feed_id == 'bb-humid'):
        client.publish("error-control","[ACK1]: "+ payload)
    elif(feed_id == 'error-control1'):
        writeSerial(str(payload))
#         buffer.pop(0)
#         sending_mess_again = False
#         failure = 0
#         published = False
#         timer = 0

In [4]:
def getPort() :
    ports = serial.tools.list_ports.comports ()
    N = len ( ports )
    commPort = "None"
    for i in range (0 , N) :
        port = ports[i]
        strPort = str(port)
        if "com0com" in strPort :
            splitPort = strPort.split (" ")
            commPort = (splitPort [0])
    return commPort

In [5]:
def processData( data ) :
    global buffer
    data = data.replace ("!", "")
    data = data.replace ("#", "")
    splitData = data.split (":") 
    print(splitData)
    buffer.append(splitData)

In [6]:
def readSerial ():
    bytesToRead = ser.inWaiting ()
    if (bytesToRead > 0) :
        global mess
        mess = mess + ser.read(bytesToRead).decode("UTF -8")
        while ("#" in mess ) and ("!" in mess ) :
            start = mess.find ("!")
            end = mess.find ("#")
            processData(mess [ start : end + 1])
            if (end == len( mess )) :
                mess = ""
            else :
                mess = mess[end +1:]

In [7]:
def writeSerial(message):
    ser.write((message).encode())

In [8]:
ser = serial.Serial(port = getPort() , baudrate =115200)

In [9]:
print(ser)

Serial<id=0x1f346ad3b50, open=True>(port='COM12', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)


In [10]:
time.time()

1633565676.0721498

In [11]:
client = MQTTClient (AIO_USERNAME , AIO_KEY)
client.on_connect = connected
client.on_disconnect = disconnected
client.on_message = message
client.on_subscribe = subscribe
client.connect()
client.loop_background()

#Implementing error control
mess = ""
buffer = []
sending_mess_again = False
failure = 0
published = False
timer = 0 
while True :
    readSerial()
    if((published == False) and (buffer != [])):
        client.publish("bb-temp", str(buffer[0][1]))
        client.publish("error-control1","[ACK2]: "+ str(buffer[0][1]))
        published = True
        timer = time.time()
    if (timer != 0) and (time.time() - timer > 3):
            sending_mess_again = True
    if sending_mess_again:
        failure += 1
        client.publish("bb-temp", str(buffer[0][1]))
        client.publish("error-control1","[ACK2]: "+ str(buffer[0][1]))
        timer = time.time()
        sending_mess_again = False
    if failure > 4:
        break

Connected to Adafruit IO!
Ket noi thanh cong ...
 Subcribe thanh cong ...
 Subcribe thanh cong ...
['TEMP', '20']
 Nhan du lieu : error-control1 [ACK2]: 20


KeyboardInterrupt: 